In [0]:
from pyspark.sql.functions import col, avg, count, round
import dlt

In [0]:
@dlt.table
def br_customers():
    df_customers = spark.read.table('dlt_catalog.raw.raw_customers_new')
    return df_customers

@dlt.table
def br_orders():
    df_orders = spark.read.table('dlt_catalog.raw.raw_orders')
    return df_orders

@dlt.table
def br_tickets():
    df_tickets = spark.read.table('dlt_catalog.raw.raw_tickets')
    return df_tickets

In [0]:
@dlt.view
def sil_customers():
    df_customers = spark.read.table('LIVE.br_customers')
    df_customers = df_customers.dropna(how='any')
    return df_customers
@dlt.view
def sil_orders():
    df_orders = spark.read.table('LIVE.br_orders')
    df_orders.where(col('status').isin('Completed', 'Pending', 'Cancelled'))
    return df_orders
@dlt.view
def sil_tickets():
    df_tickets = spark.read.table('LIVE.br_tickets')
    df_tickets = df_tickets.filter(col('resolution_time')>0)
    return df_tickets

In [0]:
from pyspark.sql.functions import avg, count

@dlt.table
def avg_res_time():
    df_cust = spark.read.table('LIVE.sil_customers')
    df_tickets = spark.read.table('LIVE.sil_tickets')
    df_res = df_tickets.join(
        df_cust,
        df_cust.customer_id == df_tickets.customer_id,
        'left'
    )
    df_res = df_res.groupBy(df_cust['customer_id'],df_cust['name']).agg(
        round(avg('resolution_time'), 2).alias('avg_res_time'))\
    .select(df_cust['customer_id'],df_cust['name'], 'avg_res_time')
    return df_res

@dlt.table
def orders_per_customer():
    df_orders = spark.read.table('LIVE.sil_orders')
    df_cust = spark.read.table('LIVE.sil_customers')
    df_order_res = df_orders.join(
        df_cust,
        df_cust.customer_id == df_orders.customer_id,
        'left'
    )
    df_order_res = df_order_res.groupBy(df_cust['customer_id']).agg(
        round(count((df_cust['customer_id'])), 2).alias('orders_per_customer')
    )
    return df_order_res